<a href="https://colab.research.google.com/github/MiguelMendozaG/CNN_2019/blob/master/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
import cv2 as cv
import time
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from google.colab import drive
drive.mount('/content/datasets/', force_remount = True)

Mounted at /content/datasets/


In [21]:
!ls "/content/datasets/My Drive/datasets"

 data_img_1.npy   data_img_3.npy  'images_CarND Udacity'
 data_img_2.npy   data_lbl.npy


In [0]:
center = np.load('/content/datasets/My Drive/datasets/data_img_1.npy')
left = np.load('/content/datasets/My Drive/datasets/data_img_2.npy')
right = np.load('/content/datasets/My Drive/datasets/data_img_3.npy')
labels = np.load('/content/datasets/My Drive/datasets/data_lbl.npy')

In [0]:
left_np = np.array(left)
center_np = np.array(center)
right_np = np.array(right)
labels_np = np.array(labels)

In [24]:
print((labels_np.shape))

(8036, 4)


In [25]:
train_perc = 0.80
#training
training_left_img = left_np[0:int(len(left_np)*train_perc)]
training_center_img = center_np[0:int(len(center_np)*train_perc)]
training_right_img = right_np[0:int(len(right_np)*train_perc)]
training_labels = labels_np[0:int(len(labels_np)*train_perc)]

#testing
testing_left_img = left_np[int(len(left_np)*train_perc):]
testing_center_img = center_np[int(len(center_np)*train_perc):]
testing_right_img = right_np[int(len(right_np)*train_perc):]
testing_labels = labels_np[int(len(labels_np)*train_perc):]

print("Size of training images array:")
print("\t Left", training_left_img.shape)
print("\t Center", training_center_img.shape)
print("\t Right", training_right_img.shape)
print ("Size of label array: ")
print("\t",  training_labels.shape)

print("\nSize of testing images array:")
print("\t Left", testing_left_img.shape)
print("\t Center", testing_center_img.shape)
print("\t Right", testing_right_img.shape)
print ("Size of label array: ")
print("\t",  testing_labels.shape)


Size of training images array:
	 Left (6428, 160, 320, 3)
	 Center (6428, 160, 320, 3)
	 Right (6428, 160, 320, 3)
Size of label array: 
	 (6428, 4)

Size of testing images array:
	 Left (1608, 160, 320, 3)
	 Center (1608, 160, 320, 3)
	 Right (1608, 160, 320, 3)
Size of label array: 
	 (1608, 4)


In [2]:
total_size = 500

data_training = np.ones((total_size, 224*224*3))*128

random_data = np.random.randint(10, size = total_size)
labels_training  = np.zeros((total_size, 10))
labels_training[np.arange(total_size), random_data] = 1


data_test = data_training[0:100]
labels_test = labels_training[0:100] 

print("Training data")
print(data_training.shape)
print(labels_training.shape)


print("Test data")
print(data_test.shape)
print(labels_test.shape)

Training data
(500, 150528)
(500, 10)
Test data
(100, 150528)
(100, 10)


In [0]:
def VGG16(img, keep_rate = 0.8, n_classes=10, batch_size = 100):
    #limpiar graficas anteriores
    #reset_graph()

    
    #Imagenes 

    #tf.summary.image("Image", img)
    
    img = tf.reshape(img, shape=[-1, 224,224,3])
    
    # Declarando las variables 
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,64])),
               'W_conv2':tf.Variable(tf.random_normal([3,3,64,64])),
               'W_conv3':tf.Variable(tf.random_normal([3,3,64,128])),
               'W_conv4':tf.Variable(tf.random_normal([3,3,128,128])),
               'W_conv5':tf.Variable(tf.random_normal([3,3,128,256])),
               'W_conv6':tf.Variable(tf.random_normal([3,3,256,256])),
               'W_conv7':tf.Variable(tf.random_normal([3,3,256,256])),
               'W_conv8':tf.Variable(tf.random_normal([3,3,256,512])),
               'W_conv9':tf.Variable(tf.random_normal([3,3,512,512])),
               'W_conv10':tf.Variable(tf.random_normal([3,3,512,512])),
               'W_conv11':tf.Variable(tf.random_normal([3,3,512,512])),
               'W_conv12':tf.Variable(tf.random_normal([3,3,512,512])),
               'W_conv13':tf.Variable(tf.random_normal([3,3,512,512])),
               'W_fc1':tf.Variable(tf.random_normal([7*7*512,4096])),
               'W_fc2':tf.Variable(tf.random_normal([4096,4096])),
               'out':tf.Variable(tf.random_normal([4096, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([64])),
              'b_conv2':tf.Variable(tf.random_normal([64])),
              'b_conv3':tf.Variable(tf.random_normal([128])),
              'b_conv4':tf.Variable(tf.random_normal([128])),
              'b_conv5':tf.Variable(tf.random_normal([256])),
              'b_conv6':tf.Variable(tf.random_normal([256])),
              'b_conv7':tf.Variable(tf.random_normal([256])),
              'b_conv8':tf.Variable(tf.random_normal([512])),
              'b_conv9':tf.Variable(tf.random_normal([512])),
              'b_conv10':tf.Variable(tf.random_normal([512])),
              'b_conv11':tf.Variable(tf.random_normal([512])),
              'b_conv12':tf.Variable(tf.random_normal([512])),
              'b_conv13':tf.Variable(tf.random_normal([512])),
              'b_fc1':tf.Variable(tf.random_normal([4096])),
              'b_fc2':tf.Variable(tf.random_normal([4096])),
              'out':tf.Variable(tf.random_normal([n_classes]))}
   
    
    
    # Declarando la arquitectura
    
    #Input: 224x224x3     Output: 224x224x64
    l1 = tf.nn.conv2d(img, weights['W_conv1'], strides=[1,1,1,1], padding='SAME')
    l1 = tf.add(l1, biases['b_conv1'])
    l1 = tf.nn.relu(l1)
    print("l1",l1.shape)
    l1 = tf.nn.dropout(l1, keep_rate)
    
    #Input: 224x224x64     Output: 224x224x64
    l2 = tf.nn.conv2d(l1, weights['W_conv2'], strides=[1,1,1,1], padding='SAME')
    l2 = tf.add(l2, biases['b_conv2'])
    l2 = tf.nn.relu(l2)    
    
    #Input: 224x224x64     Output: 112x112x64
    l2 = tf.nn.max_pool(l2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    print("l2",l2.shape)
    l2 = tf.nn.dropout(l2, keep_rate)

    
    
    
    #Input: 112x112x64     Output: 112x112x128
    l3 = tf.nn.conv2d(l2, weights['W_conv3'], strides=[1,1,1,1], padding='SAME')
    l3 = tf.add(l3, biases['b_conv3'])
    l3 = tf.nn.relu(l3)
    print("l3",l3.shape)
    #l3 = tf.nn.dropout(l3, keep_rate)
    
    #Input: 112x112x128     Output: 112x112x128
    l4 = tf.nn.conv2d(l3, weights['W_conv4'], strides=[1,1,1,1], padding='SAME')
    l4 = tf.add(l4, biases['b_conv4'])
    l4 = tf.nn.relu(l4)
    print("l4",l4.shape)
    #l4 = tf.nn.dropout(l4, keep_rate)
    
    #Input: 112x112x128     Output: 56x56x128
    l4 = tf.nn.max_pool(l4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    print("l4",l4.shape)
    #l4 = tf.nn.dropout(l2, keep_rate)
    
    
    
    
    
    #Input: 56x56x128    Output: 56x56x256
    l5 = tf.nn.conv2d(l4, weights['W_conv5'], strides=[1,1,1,1], padding='SAME')
    l5 = tf.add(l5, biases['b_conv5'])
    
    #Input: 56x56x128    Output: 56x56x256
    l6 = tf.nn.conv2d(l5, weights['W_conv6'], strides=[1,1,1,1], padding='SAME')
    l6 = tf.add(l6, biases['b_conv6'])
    
    #Input: 56x56x256    Output: 56x56x256
    l7 = tf.nn.conv2d(l6, weights['W_conv7'], strides=[1,1,1,1], padding='SAME')
    l7 = tf.add(l7, biases['b_conv7'])
    
    #Input: 56x56x256    Output: 28x28x256
    l7 = tf.nn.max_pool(l7, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    l7 = tf.nn.relu(l7)
    #print("l7",l7.shape)
    #l7 = tf.nn.dropout(l7, keep_rate)
    print("l7",l7.shape)
    
    
    
    
    #Input: 28x28x256    Output: 28x28x512
    l8 = tf.nn.conv2d(l7, weights['W_conv8'], strides=[1,1,1,1], padding='SAME')
    l8 = tf.add(l8, biases['b_conv8'])
    print("l8",l8.shape)
    
    #Input: 28x28x512    Output: 28x28x512
    l9 = tf.nn.conv2d(l8, weights['W_conv9'], strides=[1,1,1,1], padding='SAME')
    l9 = tf.add(l9, biases['b_conv9'])
    
    #Input: 28x28x512    Output: 28x28x512
    l10 = tf.nn.conv2d(l9, weights['W_conv10'], strides=[1,1,1,1], padding='SAME')
    l10 = tf.add(l10, biases['b_conv10'])
    
    #Input: 28x28x512    Output: 14x14x512
    l10 = tf.nn.max_pool(l10, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    l10 = tf.nn.relu(l10)
    #l10 = tf.nn.dropout(l10, keep_rate)
    print("l10",l10.shape)
    
    
    
    
    #Input: 14x14x512    Output: 14x14x512
    l11 = tf.nn.conv2d(l10, weights['W_conv11'], strides=[1,1,1,1], padding='SAME')
    l11 = tf.add(l11, biases['b_conv11'])
    
    #Input: 14x14x512    Output: 14x14x512
    l12 = tf.nn.conv2d(l11, weights['W_conv12'], strides=[1,1,1,1], padding='SAME')
    l12 = tf.add(l12, biases['b_conv12'])
    
    #Input: 14x14x512    Output: 14x14x512
    l13 = tf.nn.conv2d(l12, weights['W_conv13'], strides=[1,1,1,1], padding='SAME')
    l13 = tf.add(l13, biases['b_conv13'])
    
    #Input: 14x14x512    Output: 7x7x512
    l13 = tf.nn.max_pool(l13, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    l13 = tf.nn.relu(l13)
    #l13 = tf.nn.dropout(l13, keep_rate)
    print("l13",l13.shape)
    
    
    #Input: 7x7x512     Output: 4096
    fc1 = tf.reshape(l13, [-1, 7*7*512])
    fc1 = tf.nn.relu(tf.matmul(fc1, weights['W_fc1'])+biases['b_fc1'])
    #fc1 = tf.nn.dropout(fc1, keep_rate)
    print("fc1",fc1.shape)
                          
    #Input: 4096     Output: 4096
    fc2 = tf.nn.relu(tf.matmul(fc1, weights['W_fc2'])+biases['b_fc2'])
    #fc2 = tf.nn.dropout(fc2, keep_rate)
    print("fc2",fc2.shape)
    
    
    #Input: 4096     Output: 10
    output_ = tf.matmul(fc2, weights['out'])+biases['out']
    print(output_.shape)
    
    return output_

In [4]:
# Declarando las entradas y salidas
x=tf.placeholder('float',[None,224*224*3])
y=tf.placeholder('float')
keep_rate = tf.placeholder(tf.float32)
output = VGG16(x)#################################

cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

correct = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

W0813 01:49:33.827698 139663399561088 deprecation.py:506] From <ipython-input-3-e3cb3efc9fb4>:56: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0813 01:49:33.886458 139663399561088 deprecation.py:323] From <ipython-input-4-e5918da952eb>:6: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



l1 (?, 224, 224, 64)
l2 (?, 112, 112, 64)
l3 (?, 112, 112, 128)
l4 (?, 112, 112, 128)
l4 (?, 56, 56, 128)
l7 (?, 28, 28, 256)
l8 (?, 28, 28, 512)
l10 (?, 14, 14, 512)
l13 (?, 7, 7, 512)
fc1 (?, 4096)
fc2 (?, 4096)
(?, 10)


In [0]:
def evaluate(x_data, y_data, eval_batch_size):
    sess = tf.get_default_session()
    eval_i = 0
    total_cost = 0
    count = 0
    for _ in range(int(len(x_data)/eval_batch_size)):
        eval_x = x_data[eval_i:eval_i + eval_batch_size]
        eval_y = y_data[eval_i:eval_i + eval_batch_size]
        eval_cost = sess.run(cost, feed_dict = {x:eval_x, y:eval_y, keep_rate: 1})
        total_cost += eval_cost
        eval_i = eval_i + eval_batch_size
        count = count  + 1
    total_cost = total_cost / count 
    return total_cost

In [0]:
def predicted_output(x_data, y_data, k_rate = 1):
    sess = tf.get_default_session()
    y_predicted = []
    for i in range(len(x_data)):
        val = sess.run(output, feed_dict={x: x_data[i:i+1], y:y_data[i:i+1], keep_rate: k_rate, learning_rate: 0.0005})
        y_predicted.append(val)
    return y_predicted

In [0]:

def train_nn(epochs = 600, batch_size = 100, save = False):
    saver = tf.train.Saver()
    start_time = time.time()
    train_accuracy = []
    test_accuracy = []
    epoch_loss_ = []
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #writer = tf.summary.FileWriter("./Octree_logs")
        #tf.summary.FileWriter.add_graph(writer,sess.graph)
        #writer.add_graph(sess.graph)
        i_prin = 0
        for epoch in range(epochs):
            epoch_loss = 0
            i = 0
            for _ in range(int(len(data_training)/batch_size)):
                epoch_x = data_training[i:i+batch_size]
                epoch_y = labels_training[i:i+batch_size]
                #feed_dict={DNN["x"]: epoch_x,DNN["y"]: epoch_y}
                sess.run(optimizer ,feed_dict={x: epoch_x, y: epoch_y, keep_rate: 0.7})
                c = sess.run(cost, feed_dict = {x: epoch_x, y: epoch_y, keep_rate: 1})
                epoch_loss += c
                i =i+batch_size
            
            epoch_loss_.append(epoch_loss)
            train_accuracy.append(evaluate(data_training, labels_training, batch_size))
            test_accuracy.append(evaluate(data_test, labels_test, batch_size))
            
            #tf.summary.scalar('loss',epoch_loss)
            #tf.summary.scalar("train_accuracy",train_accuracy)
            #tf.summary.scalar("test_accuracy",test_accuracy)
            #summ = tf.summary.merge_all()
            
            if (epoch%20 == 0):
                pass
                #writer.add_summary(epoch_loss, epoch).eval()
            if (epoch%50 ==0):
                if isinstance(save, bool):
                    ENCname="./SVEoctree/"+str(epoch)+".ckpt"
                    saver.save(sess, ENCname)

            print("Epoch: ", epoch, "Error: ", epoch_loss_[i_prin])
            print("Train accuracy \t \t", train_accuracy[i_prin])
            print("Test accuracy \t \t", test_accuracy[i_prin])
            i_prin = i_prin + 1

    print("--- %s seconds ---" % (time.time() - start_time))
    return train_accuracy, test_accuracy, epoch_loss_

In [8]:
train_accuracy, test_accuracy, epoch_loss = train_nn(4, 100, False)

Epoch:  0 Error:  nan
Train accuracy 	 	 nan
Test accuracy 	 	 nan
Epoch:  1 Error:  nan
Train accuracy 	 	 nan
Test accuracy 	 	 nan
Epoch:  2 Error:  nan
Train accuracy 	 	 nan
Test accuracy 	 	 nan
Epoch:  3 Error:  nan
Train accuracy 	 	 nan
Test accuracy 	 	 nan
--- 86.70412540435791 seconds ---
